In [136]:
import pandas as pd
import numpy as np
import re

In [137]:
df_enroll=pd.read_csv('Cardiac Program_Archive - patient_enrollment_records.csv')

In [138]:
#print('original shape', df_enroll.shape)
#df_enroll=df_enroll[df_enroll['create_user']!='multitechvisions@gmail.com']
#print('shape after removing multitechvision rows', df_enroll.shape)

In [139]:
#Cleaning AICD
df_enroll.AICD=df_enroll.AICD.str.lower()
df_enroll=df_enroll.replace({'AICD':{'none':0, 'no':0, 'no aicd or pacemaker':0, '0' : 0,'0.25':0, '25%':0, 'o':0, '9/13/2017' : 0, 'lisinopril':0}})
df_enroll.AICD=df_enroll.AICD.fillna(9999) #to be imputed
allowed_vals=['0', 0, 9999]
df_enroll.loc[~df_enroll["AICD"].isin(allowed_vals), "AICD"] = 1
print(df_enroll['AICD'].value_counts())

9999    723
0       354
1        88
Name: AICD, dtype: int64


In [140]:
#Cleaning Acute_or_chronic
df_enroll.Acute_or_chronic=df_enroll.Acute_or_chronic.fillna(9999) #to be imputed
df_enroll.Acute_or_chronic.value_counts()

9999       604
Chronic    400
Acute      161
Name: Acute_or_chronic, dtype: int64

In [141]:
#Cleaning Diagnosis_1

def lower_errors(x):
    try:
        return x.lower()
    except:
        return ""

def find_unique_diag(df_diag_column):
    """
    Within text Diagnosis Columns, returns a list of the Unique Diagnoses,
    removing the combinations of diagnoses
    """
    all_diag=df_diag_column.apply(lambda x: lower_errors(x)).unique()
    all_diag[7].split(' , ')
    unique_diag=[]
    for diag in all_diag:
        if len(diag)==0:
            continue
        else:
            unique_diag.append(diag.split(' , '))
    flat_list = [item for sublist in unique_diag for item in sublist]
    unique_diag=pd.Series(flat_list).unique()
    return unique_diag


In [142]:
test=find_unique_diag(df_enroll.Diagnosis_1)

In [143]:
df_enroll[['patient_link','Diagnosis_1']].sample(10)


,patient_link,Diagnosis_1
331,X2KZRP2B,"Systolic CHF (EF<60%) , Atrial Fibrilation"
110,gTWeFUyb,Diastolic Heart Failure (EF >50%)
461,6AaBu2Cn,Systolic CHF (EF<60%)
1011,KlFRBHZl,NaN
92,sC611C1G,"CAD/MI , Heart Failure unspecfied"
251,Vy8HeW9P,Systolic CHF (EF<60%)
28,CKkWXWIV,CAD/MI
914,rKAXWtE1,NaN
430,wzUX64Sd,Atrial Fibrilation
222,h31anLgn,NaN


In [144]:

def dummify_diagnoses(df,unique_diag,diagnosis_col='Diagnosis_1'):
    """
    Takes Diagnoses and dummifies them for patients. If a patient has multiple
    diagnoses, will put a 1 in all relevant Diagnoses.
    The kth column is NA, no diagnosis. Maybe we will impute with the mode?
    """
    header=unique_diag.tolist().append('patient_link')
    dummy_diag=pd.DataFrame(columns=header)

    for row in range(df.shape[0]):
        pat_diag=lower_errors(df.iloc[row][diagnosis_col]).split(' , ')
        #print(pat_diag)
        dict_dummy_diag=dict(zip(unique_diag,np.zeros(len(unique_diag))))
        # dict_dummy_diag['patient_link']=df.iloc[row]['patient_link']
        #pd.DataFrame(np.zeros(len(unique_diag)).reshape(-1),columns=unique_diag)
        for diag in pat_diag:
            if diag in unique_diag:
                dict_dummy_diag[diag]=1
            else:
                continue
        tmp_dummy_diag=pd.DataFrame(dict_dummy_diag, index=[row])
        tmp_dummy_diag['patient_link']=df.iloc[row]['patient_link']
        dummy_diag = pd.concat([dummy_diag,tmp_dummy_diag], axis=0)

    return dummy_diag

In [145]:
dum_diag[dum_diag['patient_link']=='fuybpFQY']

,heart failure unspecfied,cardiomyoapthy,atrial fibrilation,cad/mi,systolic chf (ef<60%),diastolic heart failure (ef >50%),lvad,chf,angina p,patient_link
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY
12,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY
27,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY


In [146]:
pd.set_option('display.max_columns', 500)
df_enroll[df_enroll['patient_link']=='fuybpFQY']

,___Enrollment_Page,patient_link,Enrollment_Date,facilities_link,MRN,Room_number,Hospital_discharged_from,Hospital_Admit_Date,Admit_weight,Diagnosis_1,Diagnosis_2,Acute_or_chronic,AICD,status,_Discharge_Section,discharge,to_which_hospital,reason_for_dc,discharge_date,LOS,cardiac_related,Home care agency,Enrollment_Active,_Date_Tracking_Section,Session_Start_Date,Bundle_Expire,Enrollment_Timeline,Next_Enrollment_Record,___meta,create_user,creation_loc,editcount,last_user,edit_time_stamp,edit_loc,update,PatientEnrollmentID,date,time,date_time,Label,Chain_link,patient_name,Hospital_History,Previous_Enrollment_Record,Archive
1,NaN,fuybpFQY,7/31/2018,71ADiKaS,8587,604A,Coney Island Hospital,7/17/2018,125.4,"Heart Failure unspecfied , Cardiomyoapthy , At...","Parkinson's, GERD, CKD",Chronic,9999,Death,NaN,True,NaN,death,8/20/2018,0:00:00,TRUE,NaN,False,NaN,7/17/2018,10/15/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.696547, -73.953605",4,notifications@providerloop.com,8/26/2018 0:05:24,NaN,NaN,7/31/2018 10:20:58,7/31/2018,10:20:58 AM,7/31/2018 10:20:58,King David (07/31/2018 - 08/20/2018 [Death]),8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:24
12,NaN,fuybpFQY,6/8/2018,71ADiKaS,8587,805A,Mount Sinai Brooklyn,6/22/2018,131.6,"Heart Failure unspecfied , Atrial Fibrilation ...","HTN, CKD",Chronic,9999,Hospital,NaN,True,Coney Island Hospital,STROKE,7/17/2018,0:00:00,FALSE,NaN,False,NaN,6/22/2018,9/20/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.683980, -73.659496",6,notifications@providerloop.com,8/26/2018 0:05:21,NaN,NaN,6/8/2018 10:46:11,6/8/2018,10:46:11 AM,6/8/2018 10:46:11,King David (06/08/2018 - 07/17/2018 [To Hospit...,8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:21
27,NaN,fuybpFQY,5/15/2018,71ADiKaS,8587,815 -P,Mount Sinai Brooklyn,NaN,161.2,"Heart Failure unspecfied , Cardiomyoapthy , At...","HTN, CKD Stage 3, Metabolic Encephalopathy",Chronic,9999,Hospital,NaN,True,Mount Sinai Brooklyn,resident transfer to Mount Sinai in Brooklyn a...,5/17/2018,0:00:00,FALSE,NaN,False,NaN,5/15/2018,8/13/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.683980, -73.659496",4,notifications@providerloop.com,8/26/2018 0:05:17,NaN,NaN,5/15/2018 10:45:02,5/15/2018,10:45:02 AM,5/15/2018 10:45:02,King David (05/15/2018 - 05/17/2018 [To Hospit...,8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:17


In [147]:
df_enroll_diag=pd.merge(df_enroll, dum_diag, left_index=True, right_index=True)

In [148]:
df_enroll_diag

,___Enrollment_Page,patient_link_x,Enrollment_Date,facilities_link,MRN,Room_number,Hospital_discharged_from,Hospital_Admit_Date,Admit_weight,Diagnosis_1,Diagnosis_2,Acute_or_chronic,AICD,status,_Discharge_Section,discharge,to_which_hospital,reason_for_dc,discharge_date,LOS,cardiac_related,Home care agency,Enrollment_Active,_Date_Tracking_Section,Session_Start_Date,Bundle_Expire,Enrollment_Timeline,Next_Enrollment_Record,___meta,create_user,creation_loc,editcount,last_user,edit_time_stamp,edit_loc,update,PatientEnrollmentID,date,time,date_time,Label,Chain_link,patient_name,Hospital_History,Previous_Enrollment_Record,Archive,heart failure unspecfied,cardiomyoapthy,atrial fibrilation,cad/mi,systolic chf (ef<60%),diastolic heart failure (ef >50%),lvad,chf,angina p,patient_link_y
0,NaN,o9lCMyTZ,5/15/2018,71ADiKaS,8586,721 - B,New York Community Hospital,4/26/2018,131.40,Heart Failure unspecfied,HTN,Chronic,9999,Death,NaN,True,NaN,DEATH,7/22/2021,0:00:00,FALSE,NaN,False,NaN,4/26/2018,7/25/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.679693, -73.658266",3,notifications@providerloop.com,8/20/2018 1:00:15,"40.712775, -74.005973",NaN,5/15/2018 10:52:47,5/15/2018,10:52:47 AM,5/15/2018 10:52:47,King David (05/15/2018 - 07/22/2021 [Death]),8TJgy70N,"Dellernia, John",New York Community Hospital,5/15/2018 10:52:47,08/19/2018 20:00:15,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,o9lCMyTZ
1,NaN,fuybpFQY,7/31/2018,71ADiKaS,8587,604A,Coney Island Hospital,7/17/2018,125.40,"Heart Failure unspecfied , Cardiomyoapthy , At...","Parkinson's, GERD, CKD",Chronic,9999,Death,NaN,True,NaN,death,8/20/2018,0:00:00,TRUE,NaN,False,NaN,7/17/2018,10/15/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.696547, -73.953605",4,notifications@providerloop.com,8/26/2018 0:05:24,NaN,NaN,7/31/2018 10:20:58,7/31/2018,10:20:58 AM,7/31/2018 10:20:58,King David (07/31/2018 - 08/20/2018 [Death]),8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:24,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY
2,NaN,LCFBKv2z,4/12/2017,KYzNhByH,4261,331B,Maimonides MC,NaN,160.00,CAD/MI,HTN\nHx of ischemic stroke,Chronic,0,Hospital,NaN,True,Maimonides MC,Social transfer,8/17/2018,0:00:00,FALSE,NaN,False,NaN,4/12/2017,7/11/2017,NaN,NaN,NaN,update.hamilton@providerloop.com,"0.000000, 0.000000",5,joyce.giannoglou@providerloop.com,8/19/2018 22:14:00,"40.806637, -73.732759",NaN,9/10/2017 10:26:25,9/10/2017,10:26:25 AM,9/10/2017 10:26:25,Hamilton 04/12/2017 - 08/17/2018,8TJgy70N,"Taliafero, Carmen",Maimonides MC,9/10/2017 10:26:25,3/16/2018,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,LCFBKv2z
3,NaN,LnxklCqJ,10/18/2016,KYzNhByH,5600,315A,NYU Langone-Brooklyn,NaN,176.00,Heart Failure unspecfied,CHF/ AVR;\nCOPD/ HTN,Chronic,0,Hospital,NaN,True,NYU Langone-Brooklyn,Hgb 6.7,8/13/2018,0:00:00,FALSE,NaN,False,NaN,10/18/2016,1/16/2017,NaN,NaN,NaN,update.hamilton@providerloop.com,"26.503581, -80.087988",4,afeena@providerloop.com,8/16/2018 13:47:48,"40.696550, -73.953628",NaN,9/9/2017 11:46:09,9/9/2017,11:46:09 AM,9/9/2017 11:46:09,Hamilton (10/18/2016 - 8/13/2018 [Hospital]),8TJgy70N,"Jr, Martin, Kenny",NYU Langone-Brooklyn,9/9/2017 11:46:09,3/16/2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LnxklCqJ
4,NaN,pdygdpbx,5/22/2018,71ADiKaS,6907,334 - p,Maimonides MC,4/24/2018,243.50,Heart Failure unspecfied,"DM, COPD, HTN, OBESITY",Chronic,9999,Death,NaN,True,NaN,expired,8/12/2018,0:00:00,TRUE,NaN,False,NaN,4/24/2018,7/23/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.683980, -73.659496",5,notifications@providerloop.com,8/26/2018 0:10:05,NaN,NaN,5/22/2018 18:16:27,5/22/2018,6:16:27 PM,5/22/2018 18:16:27,King David (05/22/2018 - 08/12/2018 [Death]),8TJgy70N,"Wulwick, Sidney",Maimonides MC,5/22/2018 18:16:27,08/25/2018 19:10:05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pdygdpbx
5,NaN,ZqJxaCmR,8/6/2018,uqmkvwUs,12111,NaN,Brooklyn Hospital Center,12/15/2018,NaN,Systolic CHF (EF<60%),NaN,Acute,9999,Hospital,NaN,True,Mount Sinai Beth Israel,COMPLETD,8/6/2018,0:00:00,FALSE,NaN,False,NaN,12/15/2018

In [149]:
df_enroll_diag.loc[df_enroll_diag['patient_link_x']=='fuybpFQY']

,___Enrollment_Page,patient_link_x,Enrollment_Date,facilities_link,MRN,Room_number,Hospital_discharged_from,Hospital_Admit_Date,Admit_weight,Diagnosis_1,Diagnosis_2,Acute_or_chronic,AICD,status,_Discharge_Section,discharge,to_which_hospital,reason_for_dc,discharge_date,LOS,cardiac_related,Home care agency,Enrollment_Active,_Date_Tracking_Section,Session_Start_Date,Bundle_Expire,Enrollment_Timeline,Next_Enrollment_Record,___meta,create_user,creation_loc,editcount,last_user,edit_time_stamp,edit_loc,update,PatientEnrollmentID,date,time,date_time,Label,Chain_link,patient_name,Hospital_History,Previous_Enrollment_Record,Archive,heart failure unspecfied,cardiomyoapthy,atrial fibrilation,cad/mi,systolic chf (ef<60%),diastolic heart failure (ef >50%),lvad,chf,angina p,patient_link_y
1,NaN,fuybpFQY,7/31/2018,71ADiKaS,8587,604A,Coney Island Hospital,7/17/2018,125.4,"Heart Failure unspecfied , Cardiomyoapthy , At...","Parkinson's, GERD, CKD",Chronic,9999,Death,NaN,True,NaN,death,8/20/2018,0:00:00,TRUE,NaN,False,NaN,7/17/2018,10/15/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.696547, -73.953605",4,notifications@providerloop.com,8/26/2018 0:05:24,NaN,NaN,7/31/2018 10:20:58,7/31/2018,10:20:58 AM,7/31/2018 10:20:58,King David (07/31/2018 - 08/20/2018 [Death]),8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:24,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY
12,NaN,fuybpFQY,6/8/2018,71ADiKaS,8587,805A,Mount Sinai Brooklyn,6/22/2018,131.6,"Heart Failure unspecfied , Atrial Fibrilation ...","HTN, CKD",Chronic,9999,Hospital,NaN,True,Coney Island Hospital,STROKE,7/17/2018,0:00:00,FALSE,NaN,False,NaN,6/22/2018,9/20/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.683980, -73.659496",6,notifications@providerloop.com,8/26/2018 0:05:21,NaN,NaN,6/8/2018 10:46:11,6/8/2018,10:46:11 AM,6/8/2018 10:46:11,King David (06/08/2018 - 07/17/2018 [To Hospit...,8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:21,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY
27,NaN,fuybpFQY,5/15/2018,71ADiKaS,8587,815 -P,Mount Sinai Brooklyn,NaN,161.2,"Heart Failure unspecfied , Cardiomyoapthy , At...","HTN, CKD Stage 3, Metabolic Encephalopathy",Chronic,9999,Hospital,NaN,True,Mount Sinai Brooklyn,resident transfer to Mount Sinai in Brooklyn a...,5/17/2018,0:00:00,FALSE,NaN,False,NaN,5/15/2018,8/13/2018,NaN,NaN,NaN,afeena@providerloop.com,"40.683980, -73.659496",4,notifications@providerloop.com,8/26/2018 0:05:17,NaN,NaN,5/15/2018 10:45:02,5/15/2018,10:45:02 AM,5/15/2018 10:45:02,King David (05/15/2018 - 05/17/2018 [To Hospit...,8TJgy70N,"Mandel, Benjamin","Mount Sinai Brooklyn , Coney Island Hospital",7/31/2018 10:20:58,08/25/2018 19:05:17,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,fuybpFQY


In [150]:
df_enroll_diag.to_csv('clean_enrollment.csv')